## Figure 1

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

In [ ]:
core_pce = pd.read_csv("core_pce.csv")
core_pce['date'] = pd.to_datetime(core_pce['date'], dayfirst=True)
core_pce.set_index('date', inplace=True)

In [ ]:
print(core_pce.head())

In [ ]:
quarterly_pce = core_pce.resample('Q').mean()
quarterly_pce.index = quarterly_pce.index.to_period('Q')

In [ ]:
print(quarterly_pce.head())

In [ ]:
quarterly_pce['YoY Inflation (%)'] = quarterly_pce['core_pce'].pct_change(periods=4) * 100

In [ ]:
print(quarterly_pce.head(15))

In [ ]:
plot_data = quarterly_pce.loc['2008Q4':]

In [ ]:
x_axis_labels = plot_data.index.astype(str)
y_values = plot_data['YoY Inflation (%)']

In [ ]:
q4_ticks = [i for i, period in enumerate(plot_data.index) if period.quarter == 4]
q4_labels = [str(period) for i, period in enumerate(plot_data.index) if period.quarter == 4]

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(x_axis_labels, y_values, label='Core PCE Inflation', color='skyblue', linewidth=4)
plt.axhline(y=2, color='navy', linestyle='--', label="2% Target", linewidth=4)
plt.title('Figure 1: Inflation Since 2008 Q4 (Annual Rates)', fontsize=16)
plt.ylabel('Year-over-Year Inflation Rate (%)', fontsize=12)
plt.ylim(0, 2.5)

plt.gca().grid(axis='y', linestyle=':', linewidth=0.75, color='gray')

plt.xticks(ticks=q4_ticks, labels=q4_labels)
plt.gca().yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f"{x:.2f}"))

plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, fontsize=10)

plt.show()

## Figure 2

In [ ]:
rebased_core_pce = pd.read_csv("rebased_core_pce.csv")
rebased_core_pce['date'] = pd.to_datetime(rebased_core_pce['date'], dayfirst=True)
rebased_core_pce.set_index('date', inplace=True)

In [ ]:
base_date = '2009-01-01'
base_value = 100

In [ ]:
monthly_growth_factor = (1 + 0.02)**(1 / 12)

In [ ]:
rebased_core_pce['target_price_level'] = base_value * (
    monthly_growth_factor ** rebased_core_pce.index.to_series().sub(pd.Timestamp(base_date)).dt.days.div(30.44)
)

In [ ]:
quarterly_rebased = rebased_core_pce.resample('Q').mean()
quarterly_rebased.index = quarterly_rebased.index.to_period('Q')
quarterly_rebased.head()

In [ ]:
plot_data = quarterly_rebased.loc['2008Q4':]

In [ ]:
x_axis_labels = plot_data.index.astype(str)
y_values = plot_data['rebased_core_pce']
target_values = plot_data['target_price_level']

In [ ]:
q4_ticks = [i for i, period in enumerate(plot_data.index) if period.quarter == 4]
q4_labels = [str(period) for i, period in enumerate(plot_data.index) if period.quarter == 4]

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(x_axis_labels, y_values, label='Core PCE Price Index', color='skyblue', linewidth=4)
plt.plot(x_axis_labels, target_values, label='2% Target', linestyle='--', color='navy', linewidth=4)
plt.title('Figure 2: Price Level Since 2008 Q4', fontsize=16)
plt.ylabel('Index Level (2009Q1 = 100)', fontsize=12)
plt.ylim(90, 120)

plt.gca().grid(axis='y', linestyle=':', linewidth=0.75, color='gray')

plt.xticks(ticks=q4_ticks, labels=q4_labels)

plt.gca().yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f"{int(x)}"))

plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, fontsize=10)

plt.show()

## Figure 3

In [ ]:
import numpy as np

In [ ]:
data = pd.read_csv('ra_coding_data.csv')
data['date'] = pd.to_datetime(data['date'])

In [ ]:
for col in ['wti', 'copper', 'dollar']:
    data[f'dlog_{col}'] = np.log(data[col]).diff()

In [ ]:
data['d_tenyear'] = data['tenyear'].diff()

In [ ]:
data = data.dropna().reset_index(drop=True)
data.head()

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.sandwich_covariance import cov_hac

In [ ]:
start_date = '2011-06-01'
end_date = '2014-06-01'
regression_data = data[(data['date'] >= start_date) & (data['date'] <= end_date)]

In [ ]:
X = regression_data[['dlog_copper', 'dlog_dollar', 'd_tenyear']]
y = regression_data['dlog_wti']

In [ ]:
model = sm.OLS(y, X).fit()
nw_results = model.get_robustcov_results(cov_type="HAC", maxlags=5)

In [ ]:
nw_results.summary()

In [ ]:
prediction_data = data[data['date'] > '2014-06-01']

In [ ]:
coeff_copper = nw_results.params[0]
coeff_dollar = nw_results.params[1]
coeff_tenyear = nw_results.params[2]

In [ ]:
prediction_data['dlog_wti_predicted'] = (
    coeff_copper * prediction_data['dlog_copper'] +
    coeff_dollar * prediction_data['dlog_dollar'] +
    coeff_tenyear * prediction_data['d_tenyear']
)

In [ ]:
prediction_data['wti_predicted'] = prediction_data['dlog_wti_predicted'].cumsum().apply(np.exp)

In [ ]:
start_price_date = data[data['date'] <= '2014-06-01'].iloc[-1]
start_price = start_price_date['wti']

In [ ]:
prediction_data['wti_predicted'] *= start_price

In [ ]:
prediction_data['wti_actual'] = prediction_data['wti']

In [ ]:
full_data = data.copy()
full_data['wti_predicted'] = None

In [ ]:
full_data.loc[full_data['date'] > '2014-06-01', 'wti_predicted'] = prediction_data['wti_predicted']

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(full_data['date'], full_data['wti'], label='Actual WTI', color='navy', linewidth=2)
plt.plot(full_data['date'], full_data['wti_predicted'], label='Oil Demand Only', color='skyblue', linewidth=2)
plt.title('WTI Crude Estimated Demand Effect', fontsize=14)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Dollars per Barrel', fontsize=12)
plt.ylim(0, 120)
plt.legend(fontsize=12)
plt.grid(True)
plt.show()

In [ ]:
new_file = 'predicted_wti_data.csv'
full_data.to_csv(new_file, index=False)

new_file